## Prompts for locations

In [1]:
location_prompts = [
    "steampunk city with skyscrapers",
    # "cyberpunk village in Japanese rustic style",
    # "fantasy dungsseons and dragons",
    # "noir city from 1930s",
    # "StarTrek inspired spaceship",
    # "undeground mine of goblins",
    # "SuperMario style magic land plain",
    # "SuperMario style magic land beach",
]

## Generating locations 

In [2]:
from holodeck import initialize_location
from holodeck import generate_location_and_encounters
import os
from tqdm.notebook import tqdm
import traceback

import concurrent.futures
from tqdm import tqdm

def generate_location(prompt):
    location_dict, encounters_list = generate_location_and_encounters(prompt)
    if location_dict:
        try:
            location = initialize_location(location_dict, encounters_list)
            return location
        except Exception as e:
            print("Error: ", e)
            traceback.print_exc()
    else:
        print(f"GENERATING FROM '{prompt}' failed!")
        return None

locations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(executor.map(generate_location, location_prompts), total=len(location_prompts), desc="Generating locations"))
    locations += [r for r in results if r is not None]


assert len(locations) == len(location_prompts), "some locations are failed to generate"

locations

Generating locations:   0%|          | 0/1 [00:00<?, ?it/s]

<
{'name': 'Steampunk Skyline', 'description': 'A bustling city of steam-powered buildings and skyscrapers. The air is alive with the whirring of cogs and wheels and the steam of high-pressure boilers. Through the smog and the fog, you can make out the silhouettes of buildings and bridges, leading to the harbor in the north. \n', 'buildings': [{'name': 'Headquarters of the Clockwork Tower', 'description': 'The tallest building in the city and home to the ranks of the Clockwork Tower. A grand and majestic building with intricate clockwork and machinery on display, with a huge tower in the center that can be seen from far away. ', 'enterable': True}, {'name': 'The Starlight Theatre', 'description': 'An opulent theatre with a large stage and seating area, where plays, operas, and musicals can be performed. Its auditorium is lit with bright constellations of fairy lights, illuminating the elaborate decorations of the theatre.', 'enterable': True}, {'name': 'Cogwheel Café', 'description': '

Generating locations: 100%|██████████| 1/1 [00:12<00:00, 12.96s/it]

<
[{'probability': 0.1, 'description': 'As you wander the streets you come across a malfunctioning robot', 'actions': [{'type': 'critter', 'name': 'Malfunctioning Robot', 'description': 'An old, broken-down robot, its cogs and gears whirring and clanking.'}], 'trigger': {'type': 'location'}}, {'probability': 0.2, 'description': 'In the alley you stumble upon a mysterious masked figure', 'actions': [{'type': 'character', 'name': 'Mysterious Masked Figure', 'description': 'A sinister figure cloaked in a long, dark robe and wearing a peculiar, beaked mask.'}], 'trigger': {'type': 'location'}}, {'probability': 0.1, 'description': 'On the way to the theater, you see a strange contraption', 'actions': [{'type': 'item', 'name': 'Strange Contraption', 'description': 'A strange device with intricate clockwork, gears and springs.'}], 'trigger': {'type': 'locatio'}}]



[Location(name='Steampunk Skyline', description='A bustling city of steam-powered buildings and skyscrapers. The air is alive with the whirring of cogs and wheels and the steam of high-pressure boilers. Through the smog and the fog, you can make out the silhouettes of buildings and bridges, leading to the harbor in the north. \n', buildings=[Building(name='Headquarters of the Clockwork Tower', description='The tallest building in the city and home to the ranks of the Clockwork Tower. A grand and majestic building with intricate clockwork and machinery on display, with a huge tower in the center that can be seen from far away. ', enterable=True), Building(name='The Starlight Theatre', description='An opulent theatre with a large stage and seating area, where plays, operas, and musicals can be performed. Its auditorium is lit with bright constellations of fairy lights, illuminating the elaborate decorations of the theatre.', enterable=True), Building(name='Cogwheel Café', description='A 

In [3]:
b = locations[0].encounters[2
                            ]
b

Encounter(probability=0.1, description='On the way to the theater, you see a strange contraption', triggers=[], actions=[Action(type='item', item=Item(name='Strange Contraption', description='A strange device with intricate clockwork, gears and springs.'))])

## Save Locations to sqlite

In [5]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlite3 import dbapi2 as sqlite
from sqlmodel import SQLModel

# Make sure the .data directory exists
if not os.path.exists('.data'):
    os.mkdir('.data')

# Remove the database file if it exists
if os.path.exists('.data/locations.db'):
    os.remove('.data/locations.db')

# Create the engine that connects to the SQLite database
engine = create_engine('sqlite+pysqlite:///.data/locations.db', module=sqlite)

# Define a session factory that will be used to interact with the database
Session = sessionmaker(bind=engine)

# Create the tables in the database
SQLModel.metadata.create_all(engine)

# Open a session and add the locations list to it
with Session() as session:
    for location in locations:
        session.add(location)
    session.commit()

# Close the database connection
engine.dispose()


## Load Locations from sqlite

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlite3 import dbapi2 as sqlite
from sqlmodel import SQLModel, Session
from holodeck import Location

def get_saved_locations():
    # Create the engine that connects to the SQLite database
    engine = create_engine('sqlite+pysqlite:///.data/locations.db', module=sqlite)

    # Define a session factory that will be used to interact with the database
    SessionLocal = sessionmaker(bind=engine)

    # Open a session and query the locations from the database
    with SessionLocal() as session:
        locations = session.query(Location).all()

        # Close the database connection
        engine.dispose()

        return locations

[Location(name='Steampunk Skyline', id=1, description='A bustling city of steam-powered buildings and skyscrapers. The air is alive with the whirring of cogs and wheels and the steam of high-pressure boilers. Through the smog and the fog, you can make out the silhouettes of buildings and bridges, leading to the harbor in the north. \n')]

## Generate Image Prompts

In [8]:
from holodeck.gpt_text import \
        generate_object_image_prompt, \
        generate_building_image_prompt, \
        generate_location_image_prompt

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

locations_prompts = []
objects_image_prompts = []
buildings_image_prompts = []

def generate_location_images(location):
    return (location, generate_location_image_prompt(location))

def generate_object_images(location):
    prompts = []
    for o in location.objects:
        prompts.append((o, generate_object_image_prompt(o, location)))
    return prompts

def generate_building_images(location):
    prompts = []
    for b in location.all_buildings:
        prompts.append((b, generate_building_image_prompt(b, location)))
    return prompts

with ThreadPoolExecutor() as executor:
    location_image_futures = list(tqdm(executor.map(generate_location_images, locations), desc="Locations"))
    object_image_futures = [executor.submit(generate_object_images, location) for location in locations]
    building_image_futures = [executor.submit(generate_building_images, location) for location in locations]

    for location, prompt in location_image_futures:
        locations_prompts.append((location, prompt))
    for f in object_image_futures:
        for prompt in f.result():
            objects_image_prompts.append(prompt)
    for f in building_image_futures:
        for prompt in f.result():
            buildings_image_prompts.append(prompt)


img_prompts = locations_prompts + objects_image_prompts + buildings_image_prompts

obj_number = 0
for location in locations:
    obj_number += 1
    obj_number += len(location.all_buildings)
    obj_number += len(location.objects)


assert len(img_prompts) == obj_number, "not all descriptions got generated!"



Error in function generate_location_image_prompt: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 8707719488 and this is thread id 6220443648.
[SQL: SELECT building.id AS building_id, building.name AS building_name, building.description AS building_description, building.enterable AS building_enterable, building.location_id AS building_location_id 
FROM building 
WHERE ? = building.location_id]
[parameters: [{'%(4757449712 param)s': 1}]]
(Background on this error at: https://sqlalche.me/e/14/f405)
Error in function generate_location_image_prompt: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 8707719488 and this is thread id 6220443648.
[SQL: SELECT building.id AS building_id, building.name AS building_name, building.description AS building_description, building.enterable AS building_enterable, building.location_

Locations: 0it [00:00, ?it/s]


Exception: Function generate_location_image_prompt failed after 3 attempts

In [ ]:
location = locations[0]
len(location.encounters[1].triggers)


## Generate Images

In [ ]:
from holodeck.gpt_image import generate_image
from IPython.display import display, Markdown
import PIL.Image as Image
import random


random.shuffle(img_prompts)

# img_prompts = img_prompts[:9]

images = []

for obj, gen_options in tqdm(img_prompts, desc="Images"):
    prompt = gen_options['prompt']
    negative_prompt = gen_options['negative_prompt']
    image_bytes = await generate_image(prompt=prompt, negative_prompt=negative_prompt)
    image = Image.open(image_bytes)
    display(Markdown(f"### {obj.name}"))
    display(Markdown(f"""
- {prompt}
  - *negative: {negative_prompt}*
    """))
    display(image)
    image_file_name = f".images/{obj.name}.png"
    image.save(image_file_name)
    image.close()
    image_bytes.close()
    del image_bytes
    images.append((obj, prompt, image_file_name))

len(images)


## Display Images

In [ ]:
import textwrap
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
from PIL.Image import Resampling

text_width = 30
dpi = 600.0
fig_width = 3.234

columns = 4
pic_width = fig_width * 0.27


def plot_images(images):

    # Estimate the number of rows needed based on the number of images
    num_rows = (len(images) + columns - 1) // columns
    fig_height = 0
    # Get the height of the first image adjusted for scale
    with Image.open(images[0][2]) as img:
        w, h = img.size
        aspect_ratio = float(w) / h
        pic_height = pic_width / aspect_ratio

        fig_height = pic_height * num_rows * 1.6

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)

    # Define the path effect for the outline
    outline_effect = withStroke(linewidth=0.3, foreground='black')

    # Loop over the images and create a subplot for each
    for i, (obj, prompt, image_file_name) in enumerate(images):
        with Image.open(image_file_name) as image:
        
            # Resize the image
            w, h = image.size
            aspect_ratio = float(w) / h
            new_width = int(pic_width * dpi)
            new_height = int(new_width / aspect_ratio)
            with image.resize((new_width, new_height), Resampling.LANCZOS) as image_resized:
                # Create a subplot for the image
                ax = fig.add_subplot(len(images) // columns + 1, columns, i + 1)
                
                # Display the image
                ax.imshow(image_resized)

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal') # set aspect ratio to 1:1
        
        # Set the title to the obj.name
        # ax.set_title(obj.name, fontsize=4, color='magenta', pad=-10)
        
        # Wrap the prompt text to the desired width
        wrapped_prompt = textwrap.fill(prompt, width=text_width)
        wrapped_title = textwrap.fill(obj.name, width=14)
        
        # Display the wrapped prompt text below the title with black outline
        ax.text(0.08, -0.38, wrapped_prompt, ha='left', va='bottom', transform=ax.transAxes, fontsize=1.7, family='monospace', color='white', path_effects=[outline_effect])
        
        ax.text(1.0-0.08, -0.4, wrapped_title, ha='right', va='top', transform=ax.transAxes, fontsize=2.4, family='monospace', color='magenta', path_effects=[outline_effect])

    plt.subplots_adjust(wspace=0.03)

    fig.patch.set_facecolor('none')


plot_images(images)
